In [ ]:
import numpy as np
import tensorflow as tf
import torch
from tensorflow.keras.models import load_model
from tensorflow.keras import Model, layers, Input
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Load pretrained CNN model
pretrained_model = load_model("best_model_wood_92%.h5")

In [ ]:
pretrained_model.summary()

In [ ]:
# Load word embeddings
#word_embeddings = np.load('Wood_Embeddings.npy',allow_pickle=True)  # Shape: (2, 1024) for Heartwood and Sapwood

# Load label embeddings
word_embeddings = np.load("Wood_Embeddings.npy", allow_pickle=True).item()  # Dictionary with keys

# Extract embeddings and stack into a 2D array
word_embeddings = np.stack([word_embeddings['heartwood'].numpy(), 
                                   word_embeddings['sapwood'].numpy()])  # Convert tensors to numpy arrays

# Ensure word_embeddings is a Tensor
word_embeddings_tensor = tf.convert_to_tensor(word_embeddings, dtype=tf.float32)

In [ ]:
# Freeze the pretrained layers (if required)
for layer in pretrained_model.layers:
    layer.trainable = False

# Get the second-last layer's output
second_last_layer_output = pretrained_model.get_layer(index=-3).output  # Shape: (None, 128)

# Define a new trainable dense layer with 1024 neurons
dense_layer = layers.Dense(1024, activation=None, name="trainable_dense_layer")(second_last_layer_output)  # Shape: (None, 1024)

# Compute the dot product with word embeddings
dot_product = tf.matmul(dense_layer, tf.transpose(word_embeddings_tensor))  # Shape: (None, 2)

# Apply softmax activation to get predictions
predictions = layers.Softmax(name="softmax_predictions")(dot_product)

In [ ]:
# Define the new model
new_model = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model for training
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary of the new model
new_model.summary()

In [ ]:
# Load PCA data and labels
data_train = np.load('data_train_PCA.npy')  # Assuming shape (n_samples, 32 * 32 * 199)
data_test = np.load('data_test_PCA.npy')

train_labels = np.load('Wood_train_labels.npy', allow_pickle=True)  # Labels in integer format
test_labels = np.load('Wood_test_labels.npy', allow_pickle=True)

print(data_train.shape, data_test.shape, train_labels.shape, test_labels.shape )

In [ ]:
le= LabelEncoder()
y_train = le.fit_transform(train_labels)

le2= LabelEncoder()
y_test= le2.fit_transform(test_labels)

In [ ]:
# Train the model
history = new_model.fit(
    
    data_train,           # Input PCA-transformed training data
    y_train,         # Corresponding labels
    epochs=50,            # Adjust epochs based on your dataset and performance
    batch_size=32,        # Batch size
    validation_data=(data_test, y_test),  # Validation dataset
    callbacks=[           # Optional callbacks for training control
        tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
    ]
)


In [ ]:
# Evaluate the model
test_loss, test_accuracy = new_model.evaluate(data_test, y_test)

print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Save the model
new_model.save('updated_model_with_trainable_dense.h5')

In [ ]:
import matplotlib.pyplot as plt

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
# Predict class probabilities for the test set
test_predictions_prob = new_model.predict(data_test)

# Convert probabilities to predicted class labels
test_predictions = np.argmax(test_predictions_prob, axis=1)  # Assuming a multi-class problem

In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, test_predictions, target_names=["Heartwood", "Sapwood"]))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
cm = confusion_matrix(y_test, test_predictions)

# Plot confusion matrix as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=["Heartwood", "Sapwood"], yticklabels=["Heartwood", "Sapwood"])
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()
